In [ ]:
import os
os.sys.path.append('F:\KO\Python\Prj\Test\spider')

In [ ]:
import html_downloader,html_parser,html_parser_by_xpath,movie_parser_by_xpath
import threading
from lxml import etree

In [ ]:
downloader = html_downloader.HtmlDownloader()

In [ ]:
urls = ['https://www.dy2018.com/1/index.html']
urls.extend(['https://www.dy2018.com/1/index_{}.html'.format(x) for x in range(50) if x > 1])
all_movie_titles = []
lock_all_movie_titles = threading.Lock()

def download_and_get_title(url):
    global all_movie_titles
    with lock_all_movie_titles:
        print("{} start".format(url))
    html_main = downloader.download_with_para('gbk', url)
    xpath_data = etree.HTML(html_main)
    titles = xpath_data.xpath('//*[@class="ulink" and @title]/@title')
    with lock_all_movie_titles:
        all_movie_titles.extend(titles)
        print("{} done".format(url))

threads_size = 10
threads = []
for url in urls:
    url_wait_thread = True
    while url_wait_thread:
        if len(threads) < threads_size:
            new_thread = threading.Thread(target=download_and_get_title, args=(url, ))
            threads.append(new_thread)
            new_thread.start()
            url_wait_thread = False

        for i in range(len(threads)-1, -1, -1):
            threads[i].join(timeout=0.1)
            if not threads[i].is_alive():
                threads.pop(i)   
    
def filter_my_favorite_movie(title):
    tags = ['粤', '喜剧', '动画']
    for tag in tags:
        if not title.count(tag):
            return False
    return True

print('*'*50)
my_favorite_movies = filter(filter_my_favorite_movie, all_movie_titles)
for movie in list(my_favorite_movies):
    print(movie)